# Line following using the camera

In [ ]:
%config Completer.use_jedi = False
# To enable giving suggestions while pressing tab

In [ ]:
# Setting the camera direction.
import time 
from SCSCtrl import TTLServo

TTLServo.servoAngleCtrl(2, 50, -1, 150)
TTLServo.servoAngleCtrl(3, 50, -1, 150)
TTLServo.servoAngleCtrl(5, 45, 1, 150)
TTLServo.servoAngleCtrl(1, 5, -1, 150)

time.sleep(2)

TTLServo.servoStop(1)
TTLServo.servoStop(2)
TTLServo.servoStop(3)
TTLServo.servoStop(5)



In [ ]:
import traitlets
import ipywidgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg
import cv2

camera = Camera.instance(width=300, height=300)

image_widget = ipywidgets.Image()  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)


In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np


img = camera.value
# plt.imshow(img)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray_de_noised = cv2.GaussianBlur(gray, (7, 7), 0)

text = "str(center)"
coordinates = (100,100)
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
color = (255, 0, 255)
thickness = 2
img_rgb = cv2.putText(gray, text, coordinates, font, fontScale, color, thickness, cv2.LINE_AA)


fig, ax = plt.subplots(figsize=(20,20))

plt.subplot(1,5,1)
plt.imshow(img_rgb, cmap='gray')

tresh = cv2.adaptiveThreshold(gray_de_noised,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,2)
eroded = cv2.erode(gray_de_noised, None, iterations = 5)

image_enhanced = cv2.equalizeHist(gray_de_noised)
ret, thresh = cv2.threshold(eroded, 0, 255, cv2.THRESH_OTSU)
# ret, thresh = cv2.threshold(eroded, 150, 255, cv2.THRESH_BINARY)

dilated = cv2.dilate(thresh, None, iterations = 10)

plt.subplot(1,5,2)
plt.imshow(tresh, cmap='gray')

plt.subplot(1,5,3)
plt.imshow(eroded, cmap='gray')

plt.subplot(1,5,4)
plt.imshow(thresh, cmap='gray')

plt.subplot(1,5,5)
plt.imshow(dilated, cmap='gray')

ret1, thresh1 = cv2.threshold(eroded, 150, 255, cv2.THRESH_BINARY)
fig1, ax1 = plt.subplots(figsize=(5,5))
plt.hist(thresh1.ravel(),256,[0,256])
plt.show()
plt.imshow(thresh1, cmap='gray')


In [ ]:
import cv2
import numpy as np
from jetbot import Robot

# test_mode = btntoggle.value
# test_mode = False # If test mode is enabled robot will not move 

line_pos_1 = 280
line_pos_2 = 200
verticle_line_pos_1 = 50
verticle_line_pos_2 = 250
line_color_set = 0
mid_point = 150
Kp = 1
Kd = 1
Ki = 0

robot = Robot()

previous_error = 0
I = 0

def follow_line(imageInput, test_mode, dual_motor):
    global previous_error, I
    img = camera.value
    # plt.imshow(img)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_de_noised = cv2.GaussianBlur(gray, (3, 3), 0)
   

#     tresh = cv2.adaptiveThreshold(gray_de_noised,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,2)
    eroded = cv2.erode(gray_de_noised, None, iterations = 8)
    dilated = cv2.dilate(eroded, None, iterations = 5)
    ret, thresh = cv2.threshold(dilated, 0, 255, cv2.THRESH_OTSU)
#     ret, thresh = cv2.threshold(dilated, 100, 255, cv2.THRESH_BINARY)

    
    
    color_pos_1 = thresh[line_pos_1]
#     print(color_pos_1)
    color_pos_2 = thresh[line_pos_2]
    
    trimmed_line_1 = color_pos_1[verticle_line_pos_1:verticle_line_pos_2]
    trimmed_line_2 = color_pos_2[verticle_line_pos_1:verticle_line_pos_2]
    try:
        line_color_count_pos_1 = np.sum(trimmed_line_1 == line_color_set)
#         print(line_color_count_pos_1)
        line_color_count_pos_2 = np.sum(trimmed_line_2 == line_color_set)
        
        line_index_pos_1 = np.where(trimmed_line_1 == line_color_set)
#         print(line_index_pos_1)
#         print(line_index_pos_1[0])
        line_index_pos_2 = np.where(trimmed_line_2 == line_color_set)
        
        if line_color_count_pos_1 == 0:
            line_color_count_pos_1 = 1
        if line_color_count_pos_2 == 0:
            line_color_count_pos_2 = 1
            
        right_pos_1 = line_index_pos_1[0][-1] + verticle_line_pos_1
        left_pos_1 = line_index_pos_1[0][0] + verticle_line_pos_1
        center_pos_1 = int((left_pos_1 + right_pos_1)/2)
        
        right_pos_2 = line_index_pos_2[0][-1] + verticle_line_pos_1
        left_pos_2 = line_index_pos_2[0][0] + verticle_line_pos_1
        center_pos_2 = int((left_pos_2 + right_pos_2)/2)
        
        
        center = int((center_pos_1 + center_pos_2)/2)
    except Exception as err:
#         print(err)
        center = None
        pass
    thresh = cv2.cvtColor(thresh, cv2.COLOR_GRAY2RGB)
    text = str(center)
    coordinates = (100,100)
    font = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 1
    color = (0, 0, 255)
    thickness = 2
    thresh = cv2.putText(thresh, text, coordinates, font, fontScale, color, thickness, cv2.LINE_AA)
#     print(center)

   
    try:
        cv2.line(thresh, (left_pos_1, (line_pos_1 + 30)), (left_pos_1, (line_pos_1 - 30)), (255,128,64),1)
        cv2.line(thresh, (right_pos_1, (line_pos_1 + 30)), (right_pos_1, (line_pos_1 - 30)), (64,128,255),1)
        cv2.line(thresh, (0, line_pos_1), (640, line_pos_1), (255,255,64),1)
        cv2.line(thresh, (verticle_line_pos_1, 0), (verticle_line_pos_1, 640), (255,255,64),1)
        cv2.line(thresh, (verticle_line_pos_2, 0), (verticle_line_pos_2, 640), (255,255,64),1)

        
        cv2.line(thresh, (left_pos_2, (line_pos_2 + 30)), (left_pos_2, (line_pos_2 - 30)), (255,128,64),1)
        cv2.line(thresh, (right_pos_2, (line_pos_2 + 30)), (right_pos_2, (line_pos_2 - 30)), (64,128,255),1)
        cv2.line(thresh, (0, line_pos_2), (640, line_pos_2), (255,255,64),1)

        
        cv2.line(thresh, ((center - 20), int((line_pos_1 + line_pos_2)/2)), ((center + 20), int((line_pos_1 + line_pos_2)/2)), (0,0,255), 1)
        cv2.line(thresh, ((center), int((line_pos_1 + line_pos_2)/2 + 20)), ((center), int((line_pos_1 + line_pos_2)/2 - 20)), (0,0,255), 1)

    except:
        pass
    
    
    if center != None:
        direction = ""
        if 145 < center < 155 :
            direction = "Go straight"
#             robot.left_motor.value = 0.6
#             robot.right_motor.value = 0.6
        elif center < 145 :
            direction = "Turn left"
#             robot.right_motor.value = 0.9
        elif 155 < center :
            direction = "Turn right"
#             robot.left_motor.value = 0.9

        if np.all(trimmed_line_2 == trimmed_line_2[0]):
            # Stop the robot
            robot.stop()
            direction = "Stop"
#             camera.unobserve(execute, names='value')
        else:
            error = (center - mid_point)/mid_point
            P = error
            I = I + error
            D = error - previous_error
            PID_value = Kp*P + Kd*D + Ki*I
            previous_error = error
    #         print(PID_value)
            thresh = cv2.putText(thresh, str(PID_value), (190,40), font, fontScale, color, thickness, cv2.LINE_AA)


            if not test_mode:
                if dual_motor:
                    robot.left_motor.value = 0.6 + PID_value
                    robot.right_motor.value = 0.6 - PID_value
                else:
                    if PID_value > 0:
                        robot.left_motor.value = 0.6 + PID_value
                        robot.right_motor.value = 0.6 
                    elif PID_value <= 0 :
                        robot.left_motor.value = 0.6 
                        robot.right_motor.value = 0.6 - PID_value
            else:
                robot.stop()
            
        
        

#             thresh = cv2.putText(thresh, text, coordinates, font, fontScale, color, thickness, cv2.LINE_AA)

        thresh = cv2.putText(thresh, direction, (10,30), font, fontScale, color, thickness, cv2.LINE_AA)
        

        
    return thresh

In [ ]:
toggle1 = ipywidgets.ToggleButton(description='Test Mode')
display(toggle1)
test_mode = True
def on_click_1(change):
    global test_mode
    test_mode = change['new']
    
toggle1.observe(on_click_1, 'value')

toggle2 = ipywidgets.ToggleButton(description='Dual Motor Turn')
display(toggle2)

dual_motor = True
def on_click_2(change):
    global dual_motor
    dual_motor = change['new']
    
toggle2.observe(on_click_2, 'value')


In [ ]:
print(test_mode)
print(dual_motor)

In [ ]:
# print(test_mode)
# print(dual_motor)
# def run():
#     global test_mode
#     test_mode = toggle.value    
   

# toggle.observe(run, names='value')

image_widget2 = ipywidgets.Image()
def execute(change):
    
    global image_widget2
    image = change['new']
    image_widget.value = bgr8_to_jpeg(image)
    image_widget2.value = bgr8_to_jpeg(follow_line(image,test_mode, dual_motor))

display(image_widget2)



execute({'new': camera.value})

camera.unobserve_all()

camera.observe(execute, names='value')
   

In [ ]:
camera.unobserve(execute, names='value')

In [ ]:
camera.stop()

In [ ]:
robot.stop()

In [ ]:
import time
robot.forward(0.3)
time.sleep(1)
robot.stop()

In [ ]:
import time
robot.left_motor.value = 0.5
robot.right_motor.value = -0.5
# time.sleep(15)
# robot.stop()

In [ ]:
robot.stop()

In [ ]:
a = np.arange(10)
b = np.where(a == 5)
print(a)
print(b)
print(color_pos_1)

In [ ]:
img = camera.value
x = follow_line(img)

# Center coordinates
center_coordinates = (92, 280)
 
# Radius of circle
radius = 20
  
# Blue color in BGR
color = (255, 0, 0)
  
# Line thickness of 2 px
thickness = 2
  
# Using cv2.circle() method
# Draw a circle with blue line borders of thickness of 2 px
x = cv2.circle(x, center_coordinates, radius, color, thickness)

plt.imshow(x)

In [ ]:
print(color_pos_1)

In [ ]:
btntoggle = ipywidgets.ToggleButton(
    value=False,
    description='Click me',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

print("asf")
display(btntoggle)
def toggle_click(change):
    print("safa")
    new_value = change['new']
    print(new_value)
btntoggle.unobserve_all()
btntoggle.observe(toggle_click, names='value')
# print(btntoggle.value)

In [ ]:
import ipywidgets
from IPython.display import display

toggle = ipywidgets.ToggleButton(description='click me')

def on_click(change):
    print("safa")
    print(change['old'])
    display("saf")

toggle.observe(on_click, names=['value'])
display(toggle)

In [ ]:
import ipywidgets as w
toggle = w.ToggleButton(description='click me')

out = w.Output(layout=w.Layout(border = '1px solid black'))

def fun(obj):
    with out:
#         if obj['new']:  
            display("asf")
#         else:
#             out.clear_output()

toggle.observe(fun, 'value')
display(toggle)
display(out)

In [ ]:
toggle.value